# Atividade 2 - Visão Computacional

O entregável de toda esta atividade vai ser um código-fonte em *Python*. 

Encorajamos vocês a fazerem vídeos demonstrando o resultado e a postar (pode ser privadamente) no YouTube



Você deve ter uma folha com o padrão anexo. 
*Dica:* Se não tiver, é possível fazer também com um tablet ou *smartphone*
 
<img src="folha_atividade.png" width=300>

In [3]:
%matplotlib inline
import cv2
from matplotlib import pyplot as plt
import numpy as np
import time as t
import sys
import math
print ("OpenCV Version : %s " % cv2.__version__)

from ipywidgets import widgets, interact, interactive, FloatSlider, IntSlider

import auxiliar as aux

if (sys.version_info > (3, 0)): 
    # Modo Python 3
    import importlib
    importlib.reload(aux) # Para garantir que o Jupyter sempre relê seu trabalho
else:
    # Modo Python 2
    reload(aux)

OpenCV Version : 3.4.2 


## Parte 1 - calibração

Ouça a explicação do professor sobre o modelo de câmera *pinhole*  e desenhe a medida $f$ que separa o plano focal da pupila da câmera

Detalhe como calculou $f$

In [2]:
ho = 13.9 #distância dos centros dos círculos em centímetros
hi = 627 #distância dos centros dos círculos na tela em pixels
d = 21.2 #distância da tela até o final do teclado em centímetros

f = (hi*d)/ho
f

956.2877697841726

## Parte 2

Modifique um dos exemplos `draw_circles_video.py` ou `videoplay.py` para passar a ler dados da webcam e identificar o círculo magenta e o círculo ciano, usando o `inRange`

In [34]:
import cv2
import numpy as np

cap = cv2.VideoCapture(0)

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    
    # Our operations on the frame come here
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    minm = np.array([142, 30, 30])
    maxm = np.array([192, 255, 255])
    
    maskm = cv2.inRange(hsv, minm, maxm) 
    
    minc = np.array([92, 40, 40])
    maxc = np.array([122, 255, 255])
    
    maskc = cv2.inRange(hsv, minc, maxc) 

    # Display the resulting frame
#    cv2.imshow('mask', maskm)
#    cv2.imshow('mask', maskc)

    masks = cv2.bitwise_or(maskm, maskc)
    juncao = cv2.bitwise_and(frame, frame, mask=masks)
    
    juncao_rgb = (juncao, cv2.COLOR_HSV2RGB)
    
    cv2.imshow('frame', juncao)
#    cv2.imshow('frame', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

## Parte 3

Assumindo que a folha se mantém sempre paralela ao plano de imagem da câmera, imprima a distância entre a folha e sua câmera

## Obs: Partes 3, 4 e 5 juntas ##

In [37]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import time

# If you want to open a video, just change this path
#cap = cv2.VideoCapture('hall_box_battery.mp4')

# Parameters to use when opening the webcam.
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

lower = 0
upper = 1

# Returns an image containing the borders of the image
# sigma is how far from the median we are setting the thresholds
def auto_canny(image, sigma=0.33):
    # compute the median of the single channel pixel intensities
    v = np.median(image)

    # apply automatic Canny edge detection using the computed median
    lower = int(max(0, (1.0 - sigma) * v))
    upper = int(min(255, (1.0 + sigma) * v))
    edged = cv2.Canny(image, lower, upper)

    # return the edged image
    return edged



while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # A gaussian blur to get rid of the noise in the image
    blur = cv2.GaussianBlur(gray,(9,9),0)
    #blur = gray
    # Detect the edges present in the image
    bordas = auto_canny(blur)


    circles = []
#    centros = []

    # Obtains a version of the edges image where we can draw in color
    bordas_color = cv2.cvtColor(bordas, cv2.COLOR_GRAY2BGR)

    # HoughCircles - detects circles using the Hough Method. For an explanation of
    # param1 and param2 please see an explanation here http://www.pyimagesearch.com/2014/07/21/detecting-circles-images-using-opencv-hough-circles/
    circles = None
    circles=cv2.HoughCircles(bordas,cv2.HOUGH_GRADIENT,2,40,param1=50,param2=100,minRadius=5,maxRadius=60)

    if circles is not None:
        circles = np.uint16(np.around(circles))

        for i in circles[0,:]:
            # draw the outer circle
            # cv2.circle(img, center, radius, color[, thickness[, lineType[, shift]]])
            cv2.circle(bordas_color,(i[0],i[1]),i[2],(0,255,0),2)
            # draw the center of the circle
            cv2.circle(bordas_color,(i[0],i[1]),2,(0,0,255),3)
            # cv2.line(img, pt1, pt2, color[, thickness[, lineType[, shift]]])
#            centros.append([i[0], i[1]])
            
        if len(circles[0]) == 2:
            x1, y1, r1 = circles[0][0]
            x2, y2, r2 = circles[0][1]
            
            x = int(x2)-int(x1)
            y = int(y2)-int(y1)
                                       
            d_h1 = math.sqrt(x**2 + y**2)
            
            # f e h0 q foram definidos na parte 1
            f = 956.2877697841726
            ho = 13.9
            
            d = (f*ho)/d_h1
            
            a = math.degrees(math.atan(y/x))
            
            cv2.line(bordas_color, (x1, y1), (x2, y2), (255,0,0), 5)
        
        
            #cv2.putText(img, text, org, fontFace, fontScale, color[, thickness[, lineType[, bottomLeftOrigin]]])
            font = cv2.FONT_HERSHEY_SIMPLEX
            cv2.putText(bordas_color,'Distance: {0}'.format(d),(0,50), font, 1,(255,255,255),2,cv2.LINE_AA)

            cv2.putText(bordas_color,'Angle: {0}'.format(a),(0,90), font, 1,(255,255,255),2,cv2.LINE_AA)

    # Display the resulting frame
    cv2.imshow('Detector de circulos',bordas_color)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

ZeroDivisionError: division by zero

## Parte 4

Trace uma linha entre os centros do círculo magenta e do círculo ciano.

Imprima na tela o ângulo entre esta linha e a horizontal

## Parte 5

Usando transformada de Hough, desenhe um círculo sobre o círculo ciano e outro sobre o círculo magenta.

**Desafio bônus**: ser capaz de eliminar circulos espúrios (aqueles que não são os da folha)

## Parte 6

Usando `SIFT`, identifique o escrito *Insper* na folha

In [25]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
from math import pi
import matplotlib.cm as cm

In [29]:
# Número mínimo de pontos correspondentes
MIN_MATCH_COUNT = 5

imagem = cv2.imread('insper.jpg',0) # Imagem do cenario - puxe do video para fazer isto



In [30]:

cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

# Cria o detector SIFT
sift = cv2.xfeatures2d.SIFT_create()

# Encontra os pontos únicos (keypoints) nas duas imagems
kp1, des1 = sift.detectAndCompute(imagem ,None)

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Imagem de saída
    out = frame_rgb.copy()
    
    kp2, des2 = sift.detectAndCompute(imagem, None)
    
    cv2.imshow("captura", out)
    
    # Configurações do algoritmo FLANN que compara keypoints e ver correspondências - não se preocupe com isso
    FLANN_INDEX_KDTREE = 0
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    search_params = dict(checks = 50)

    # Configura o algoritmo de casamento de features que vê *como* o objeto que deve ser encontrado aparece na imagem
    flann = cv2.FlannBasedMatcher(index_params, search_params)

    # Tenta fazer a melhor comparacao usando o algoritmo
    matches = flann.knnMatch(des1,k=2)

    # store all the good matches as per Lowe's ratio test.
    good = []
    for m,n in matches:
        if m.distance < 0.7*n.distance:
            good.append(m)


    print("Matches", len(good))
            
    if len(good)>MIN_MATCH_COUNT:
        # Separa os bons matches na origem e no destino
        src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
        dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)


        # Tenta achar uma trasformacao composta de rotacao, translacao e escala que situe uma imagem na outra
        # Esta transformação é chamada de homografia 
        # Para saber mais veja 
        # https://docs.opencv.org/3.4/d9/dab/tutorial_homography.html
        M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC,5.0)
        matchesMask = mask.ravel().tolist()



        h,w = imagem.shape
        # Um retângulo com as dimensões da imagem original
        pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)

        # Transforma os pontos do retângulo para onde estao na imagem destino usando a homografia encontrada
        dst = cv2.perspectiveTransform(pts,M)


        # Desenha um contorno em vermelho ao redor de onde o objeto foi encontrado
        img2b = cv2.polylines(frame,[np.int32(dst)],True,(255,0,0),3, cv2.LINE_AA)

    else:
        print("Not enough matches are found - %d/%d" % (len(good),MIN_MATCH_COUNT))
        matchesMask = None

    draw_params = dict(matchColor = (0,255,0), # draw matches in green color
                       singlePointColor = None,
                       matchesMask = matchesMask, # draw only inliers
                       flags = 2)
    
    # Display the resulting frame
    cv2.imshow('SIFT',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

Matches 0
Not enough matches are found - 0/5
Matches 0
Not enough matches are found - 0/5
Matches 0
Not enough matches are found - 0/5
Matches 0
Not enough matches are found - 0/5
Matches 0
Not enough matches are found - 0/5
Matches 0
Not enough matches are found - 0/5
Matches 0
Not enough matches are found - 0/5
Matches 0
Not enough matches are found - 0/5
Matches 0
Not enough matches are found - 0/5
Matches 0
Not enough matches are found - 0/5
Matches 0
Not enough matches are found - 0/5
Matches 0
Not enough matches are found - 0/5
Matches 0
Not enough matches are found - 0/5
Matches 0
Not enough matches are found - 0/5
Matches 0
Not enough matches are found - 0/5
Matches 0
Not enough matches are found - 0/5
Matches 0
Not enough matches are found - 0/5
Matches 0
Not enough matches are found - 0/5
Matches 0
Not enough matches are found - 0/5
Matches 0
Not enough matches are found - 0/5
Matches 0
Not enough matches are found - 0/5
Matches 0
Not enough matches are found - 0/5
Matches 0


In [28]:
!ls

Atividade2.ipynb             folha_atividade.png
RinTinTin.jpg                hall_box_battery1.jpg
SIFT Features.ipynb          hall_box_battery_1024.mp4
__pycache__                  houghlines.py
aula2_OpenCV_Filtragem.ipynb insper-nome.png
auxiliar.py                  insper.jpg
box.png                      insperlogo.jpeg
box_in_scene.png             sift.py
coke-cans.jpg                videoplay.py
draw_circles_video.py        videoplay_canny.py
folha_atividade.pdf


In [38]:
cap.release()
cv2.destroyAllWindows()

In [5]:
!pwd 

/Users/gabrielacaruso/Documents/Insper/3 Semestre/Robótica Computacional/robot19/aula_02
